In [9]:
import pyttsx3

engine = pyttsx3.init()
engine.setProperty('rate', 180)
engine.say('您好，我是智能汽车辅助系统，您被绑定的驾驶员疑似发生酒后驾驶行为，请及时确认')
engine.runAndWait()

engine.save_to_file('您好，我是智能汽车辅助系统，您被绑定的驾驶员疑似发生酒后驾驶行为，请及时确认', 'test1.mp3')

In [1]:
import serial
import serial.tools.list_ports
import re
from paho.mqtt import client as mqtt_client
import pyttsx3

LAST_STATUS = False
LAST_DATA = 0


class Contral:
    def __init__(self):
        self.ser = serial.Serial(
            Contral.findPort(), timeout=1, baudrate=115200)
        self.client_id = 'contral_client'
        self.broker = '119.91.198.5'
        self.port = 1883
        self.client = self.connect_mqtt()
        self.topic = '/call'
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 180)
        self.dangerVoice = '喝酒不开车，开车不喝酒。'
        if self.is_open():
            print('Serial port is open')
            self.run()
        else:
            print('Serial port is not open')

    def __del__(self):
        self.ser.close()
        self.client.disconnect()

    def publish(self):
        message = '{"contacts":"18273623931"}'
        self.client.publish(self.topic, message)

    def connect_mqtt(self):
        def on_connect(client, userdata, flags, rc):
            if rc == 0:
                print("Connected to MQTT Broker!")
            else:
                print("Failed to connect, return code %d\n", rc)

        client = mqtt_client.Client(self.client_id)
        client.on_connect = on_connect
        client.connect(self.broker, self.port)
        return client

    def is_open(self):
        return self.ser.is_open

    def run(self):
        while True:
            line = self.ser.readline()
            if line:
                line.decode('ascii', 'ignore')
                line = str(line, 'utf-8')
                line = line.replace('\r\n', '')
                data = line[line.find(':') + 1:]
                expression = ''.join(re.findall(r'[A-Za-z]', line))
                if expression == 'alcohol':
                    print('alcohol-->', float(data))
                    if float(
                            data
                    ) > 200 and LAST_STATUS is False and LAST_DATA < float(
                            data):
                        LAST_STATUS = True
                        LAST_DATA = float(data)
                        self.engine.say(self.dangerVoice)
                        self.engine.runAndWait()
                        self.publish()
                        print('publish')
                    elif float(data) < 200:
                        LAST_STATUS = False
                        LAST_DATA = float(data)
                    elif float(data) == LAST_DATA:
                        pass
                    else:
                        LAST_DATA = float(data)

    def findPort():
        ports = serial.tools.list_ports.comports()
        for each in ports:
            print(each.description)
            if 'CP210x USB to UART' in each.description:
                return each.device


In [2]:
con = Contral()

Silicon Labs CP210x USB to UART Bridge (COM11)


D:\Softwares\Miniconda3\envs\deeplearning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Softwares\Miniconda3\envs\deeplearning\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Softwares\Miniconda3\envs\deeplearning\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


alcohol--> 0.0
alcohol--> 0.0
alcohol--> 0.0
alcohol--> 33.0
alcohol--> 97.0
alcohol--> 160.0
alcohol--> 224.0
alcohol--> 288.0
alcohol--> 347.0
alcohol--> 400.0
alcohol--> 400.0
alcohol--> 400.0
alcohol--> 400.0
alcohol--> 400.0
alcohol--> 395.0
alcohol--> 331.0
alcohol--> 267.0
alcohol--> 208.0
alcohol--> 144.0
alcohol--> 80.0
alcohol--> 17.0
alcohol--> 0.0
alcohol--> 0.0
alcohol--> 0.0
alcohol--> 0.0
alcohol--> 0.0
